In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

In [2]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
split = 1

In [4]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/combosciplex/adata_with_predictions_{split}.h5ad")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [5]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [6]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [7]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [8]:
adata_pred_test = adata_preds[adata_preds.obs["split"]=="test"]
adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds[adata_preds.obs["split"]=="ood"]
adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

/tmp/ipykernel_1395211/2350350615.py:2: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
/tmp/ipykernel_1395211/2350350615.py:18: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]


In [9]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [10]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [11]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [12]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [13]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [14]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7583311370440892,
 'decoded_ood_sinkhorn_div_1': 104.91385105678013,
 'decoded_ood_sinkhorn_div_10': 70.87678473336356,
 'decoded_ood_sinkhorn_div_100': 24.199761526925222,
 'decoded_ood_e_distance': 47.56255558561155,
 'decoded_ood_mmd': 0.40828136035374235}

In [15]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7544632653395335,
 'decoded_test_e_distance': 47.468259798773005,
 'decoded_test_mmd_distance': 0.4137947695950667}

In [16]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7583311370440892,
 'decoded_ood_sinkhorn_div_1': 104.91385105678013,
 'decoded_ood_sinkhorn_div_10': 70.87678473336356,
 'decoded_ood_sinkhorn_div_100': 24.199761526925222,
 'decoded_ood_e_distance': 47.56255558561155,
 'decoded_ood_mmd': 0.40828136035374235}

In [17]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -0.16320323092596872,
 'encoded_ood_sinkhorn_div_1': 28.561551366533553,
 'encoded_ood_sinkhorn_div_10': 24.534843581063406,
 'encoded_ood_sinkhorn_div_100': 23.112079892839706,
 'encoded_ood_e_distance': 45.89178975814783,
 'encoded_ood_mmd': 0.372120948774474}

In [18]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.5687286172594342,
 'deg_ood_sinkhorn_div_1': 22.325497218540736,
 'deg_ood_sinkhorn_div_10': 14.003204550061907,
 'deg_ood_sinkhorn_div_100': 13.36653743471418,
 'deg_ood_e_distance': 26.604476978617836,
 'deg_ood_mmd': 0.4617248134953635}

In [19]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.5871160055200259,
 'deg_test_sinkhorn_div_1': 23.177490572134655,
 'deg_test_sinkhorn_div_10': 14.496344804763794,
 'deg_test_sinkhorn_div_100': 13.409663081169128,
 'deg_test_e_distance': 26.661953989414798,
 'deg_test_mmd': 0.44162969539562863}

In [20]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7544632653395335,
 'decoded_test_e_distance': 47.468259798773005,
 'decoded_test_mmd_distance': 0.4137947695950667}

In [21]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7583311370440892,
 'decoded_ood_sinkhorn_div_1': 104.91385105678013,
 'decoded_ood_sinkhorn_div_10': 70.87678473336356,
 'decoded_ood_sinkhorn_div_100': 24.199761526925222,
 'decoded_ood_e_distance': 47.56255558561155,
 'decoded_ood_mmd': 0.40828136035374235}

In [22]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.5687286172594342,
 'deg_ood_sinkhorn_div_1': 22.325497218540736,
 'deg_ood_sinkhorn_div_10': 14.003204550061907,
 'deg_ood_sinkhorn_div_100': 13.36653743471418,
 'deg_ood_e_distance': 26.604476978617836,
 'deg_ood_mmd': 0.4617248134953635}

In [23]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7544632653395335,
 'decoded_test_e_distance': 47.468259798773005,
 'decoded_test_mmd_distance': 0.4137947695950667}

In [24]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -0.5184901296834094,
 'encoded_test_sinkhorn_div_1': 27.951833486557007,
 'encoded_test_sinkhorn_div_10': 23.561344703038532,
 'encoded_test_sinkhorn_div_100': 22.2146254380544,
 'encoded_test_e_distance': 44.141308159933125,
 'encoded_test_mmd': 0.383746054644386}

In [25]:
ood_metrics_encoded

{'Givinostat+Dasatinib': {'r_squared': 0.7096127867698669,
  'sinkhorn_div_1': 7.011560440063477,
  'sinkhorn_div_10': 3.322206497192383,
  'sinkhorn_div_100': 2.246318817138672,
  'e_distance': 4.23840041103893,
  'mmd': 0.24613881},
 'Givinostat+SRT1720': {'r_squared': 0.5914133787155151,
  'sinkhorn_div_1': 7.524575233459473,
  'sinkhorn_div_10': 3.9772605895996094,
  'sinkhorn_div_100': 2.984194755554199,
  'e_distance': 5.743551044620476,
  'mmd': 0.2560083},
 'Panobinostat+Dasatinib': {'r_squared': -1.1322658061981201,
  'sinkhorn_div_1': 53.164649963378906,
  'sinkhorn_div_10': 48.90763473510742,
  'sinkhorn_div_100': 47.201011657714844,
  'e_distance': 94.02335283147708,
  'mmd': 0.51081914},
 'Panobinostat+PCI-34051': {'r_squared': -1.2073190212249756,
  'sinkhorn_div_1': 60.13479232788086,
  'sinkhorn_div_10': 55.326324462890625,
  'sinkhorn_div_100': 53.203887939453125,
  'e_distance': 105.89463072406123,
  'mmd': 0.52954537},
 'Panobinostat+SRT2104': {'r_squared': -1.262223

In [26]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/chemcpa"

In [27]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
